# Amazon SageMaker XGBoost アルゴリズムによる多クラス分類
---
## 目次

1. [はじめに](#はじめに)
2. [データの準備](#データの準備)
  1. [データのダウンロード](#データのダウンロード)
  2. [データの加工とS3へのアップロード](#データの加工とS3へのアップロード)
3. [XGBoostモデルの学習](#XGBoostモデルの学習)
4. [推論](#推論)
  1. [XGBoostをインストールして推論](#XGBoostをインストールして推論)
  2. [エンドポイントを作成して推論](#エンドポイントを作成して推論)
---
## はじめに


このnotebookでは、[deeplearning.net](http://deeplearning.net/)で公開されているMNIST (Modified National Institute of Standards and Technology)の手書き数字データセットを利用して、書かれている数字を認識します。MNISTのサンプルを示します。
![MNISTサンプル](./images/mnist.png "サンプル")

MNISTには、学習用に60,000枚、テスト用に10,000枚のラベル付き画像が用意されており、画像の解像度は28x28です。各画像にどの数字が書かれているかを判別するために、多クラス分類器として有用なXGBoostを利用します。

---
## データの準備

### データのダウンロード
まず[deeplearning.net](http://deeplearning.net/)からデータをダウンロードします。ダウンロードされるファイルはバイナリで、ファイルを読み込むと`train_set, valid_set, test_set`の3つを得ることができます。各セットには、**画像を表す配列**と**ラベルを表す配列をリスト**として含んでいます。画像枚数は、それぞれ50,000枚、10,000枚、10,000枚です。したがって、`train_set`には50,000x784の画像を表す配列と、50,000のラベルを表す配列が入っていることになります。ここで784とは28x28の画像をベクトル化したときの次元数を表します。

In [ ]:
%%time

import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
from sagemaker import get_execution_role
import pickle, gzip, numpy, urllib.request, json

# Load the dataset
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
f.close()

### データの加工とS3へのアップロード

データをダウンロードしたら、SageMaker XGBoostが読み込めるファイル形式に加工します。SageMaker XGBoostが読み込めるファイルはcsvまたはlibsvmです。ここではlibsvmの形式に加工します。libsvm形式とは以下のような形式です。
```
3 1:0.0 2:0.0 3:0.0 4:0.0 5:0.0 6:0.0 7:0.0 8:0.0 9:0.0 10:0.0 11:0.0 12:0.0 13:0.0 ...
```
最初の数字が`ラベル`を表し、以降は`特徴のindex:特徴量`が並びます。以下のセルでは`to_libsvm`の関数がlibsvm形式への変換を行います。

以降、高性能な学習用インスタンスを立ち上げてモデルを学習するため、それらのインスタンスがアクセスできるようにファイルをS3に置く必要があります。
各セットを`data.train, data.valid, data.test`というファイル名で保存したら、 `sess.upload_data`を利用してS3にアップロードします。S3のアップロード先は、バケット名が`default_backet()`によって自動設定される`sagemaker-{region}-{AWS account ID}`で、prefixが`notebook/xgboost/mnist`となります。バケット名も自由に設定できますが、世界中で唯一の名前となるような設定が必要です。


In [ ]:
%%time

import struct
import io
import boto3
import pickle
import gzip
import sagemaker

sess = sagemaker.Session()
role = get_execution_role()
bucket = sess.default_bucket()
prefix = 'notebook/xgboost/mnist'
def to_libsvm(f_name, labels, values):
    with open(f_name,'w',encoding = 'utf-8') as f:
        content = '\n'.join(['{} {}'.format(label, ' '.join(['{}:{}'.format(i + 1, el) for i, el in enumerate(vec)])) for label, vec in
          zip(labels, values)])
        f.write(content)        

with gzip.open('mnist.pkl.gz', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    train_set, valid_set, test_set = u.load()

to_libsvm('data.train', train_set[1], train_set[0])
to_libsvm('data.valid', valid_set[1], valid_set[0])
to_libsvm('data.test', test_set[1], test_set[0])

train_input = sess.upload_data(
        path='data.train', 
        key_prefix=prefix)
valid_input = sess.upload_data(
        path='data.valid', 
        key_prefix=prefix)
test_input = sess.upload_data(
        path='data.test', 
        key_prefix=prefix)


## XGBoostモデルの学習

ファイルをS3に置くことができたら学習を始めます。まず学習をするためのコンテナのイメージを指定する必要があります。各アルゴリズムに対応するイメージのリストは以下にあります。https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

学習を行うまでの流れは以下の通りです。
1. sagemakerのEstimatorを指定する。学習に使用するインスタンスとその数、モデルを出力するS3のフォルダを指定します。
1. ハイパーパラメータを指定します。ここでは多クラス分類なので、`objective='multi:softmax',num_class=10`の指定が必要です。
1. S3のファイルパスを与えてfitします。

In [ ]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest',
             'ap-northeast-1': '501404015308.dkr.ecr.ap-northeast-1.amazonaws.com/xgboost:latest'}

training_job_name = 'DEMO-xgboost-classification' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.4xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                   sagemaker_session=sess)
xgb.set_hyperparameters(eta=0.1,
                        objective='multi:softmax',
                        num_class=10,
                        num_round=25)

xgb.fit({'train': train_input, 'validation': valid_input}, job_name = training_job_name)

## 推論

推論を行う方法として主に以下の２種類の方法があります。
- **ノートブックインスタンスにxgboostをインストールしてローカルで推論する**  
ノートブックインスタンスで推論を行う場合、推論用のインスタンスを立ち上げる必要がありません。開発段階において様々なモデルを試したいときに、推論用のインスタンスを立ち上げることなく、結果を知りたい場合に便利です。

- **エンドポイントを作成して推論する**  
サービスとして推論を利用するとき、サービスの負荷などを考慮して、ノートブックインスタンスよりも高性能なインスタンスで推論したい場合があります。その場合は、エンドポイントを作成して推論するほうが良いです。

### XGBoostをインストールして推論

まずXGBoostをcondaからインストールします。インストールが終わると、SageMaker XGBoostで学習したモデルS3からダウンロードしてXGBoostに読み込ませます。これで予測をする準備が整いました。

ローカルにある`data.test`をXGBoostが扱う`DMatrix`形式にして読み込み、`predict`するとテストデータに対する予測結果を得ることができます。

In [ ]:
# ノートブックインスタンスにxgboostをインストール
try:
    import xgboost as xg
    print('XGboost {} has already been installed. '.format(xg.__version__))
except:
    !source activate python3 && conda install -y  -c conda-forge xgboost
    from IPython.display import clear_output
    clear_output()
    import xgboost as xg
    print('XGboost {} is installed. '.format(xg.__version__))
    
    
# S3にあるファイルをダウンロードして解凍
import boto3
import botocore
s3 = boto3.resource('s3')
model_location =prefix +'/output/' + training_job_name + '/output/model.tar.gz'
print("The model is saved at {}".format('s3://' + bucket +'/'+model_location))
try:
    s3.Bucket(bucket).download_file(model_location, 'model.tar.gz')
    !tar -zxvf model.tar.gz
    print('Downloading and extracting the model are done.')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
        import sys
        sys.exit()
    else:
        raise

# モデルのロード
import pickle as pkl
xgb_model = pkl.load(open('xgboost-model','rb'))
print('Loading model is done.')

#テストデータを読み込んで手書き文字(0-9)を分類
dtest = xg.DMatrix('data.test')
prediction = xgb_model.predict(dtest)

### 予測結果の確認

以下ではランダムに1枚の画像を選んで、その画像に対する識別結果と画像を表示します。選ぶ画像を変えたい場合は、以下のセルを再実行してください。

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm

select_row = np.random.choice(range(len(test_set[1])))
feature = test_set[0][select_row]
pred_result = int(prediction[select_row])
plt.imshow(feature.reshape(28, 28), cmap=cm.gray_r)
print('Prediction: {}'.format(pred_result))

## エンドポイントを作成して推論
**ハンズオンではオプションとします。エンドポイント作成については以降でも実施します。  
もしエンドポイントを作成した場合は課金が発生しますので、後述の方法でエンドポイントの削除を忘れないようにしましょう。**


### エンドポイントの作成
SageMakerを用いると、インスタンス数とタイプを指定して`deploy`するだけでエンドポイントを作成できます。

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### エンドポイントによる推論

In [ ]:
%matplotlib inline
#エンドポイントが受け取るcontentを指定します。
from sagemaker.predictor import csv_serializer
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

#ランダムにテストデータを選んで推論します。
#予測の部分はpredictの部分だけです。
select_row = np.random.choice(range(len(test_set[1])))
feature = test_set[0][select_row]
pred_result = xgb_predictor.predict(test_set[0][select_row])
pred_result = int(float(pred_result.decode("utf-8")))
plt.imshow(feature.reshape(28, 28), cmap=cm.gray_r)
print('Prediction: {}'.format(pred_result))

### エンドポイントの削除

In [ ]:
xgb_predictor.delete_endpoint()